<a href="https://colab.research.google.com/github/atharvaballa/Stationery-Store-Management-System-with-Receipt-Generation-and-Data-Encryption/blob/main/Creating_Payment_Receipts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import datetime
import os
import sqlite3
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from cryptography.fernet import Fernet
from google.colab import drive

# Generate a Fernet key (store this securely for later decryption)
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Database functions
def create_tables():
    """Create the database tables if they don't exist."""
    # Database connection and cursor
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    # Create items table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        price REAL NOT NULL
    )
    ''')
    # Create transactions table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        transaction_id TEXT NOT NULL,
        total_amount REAL NOT NULL,
        date TEXT NOT NULL,
        payment_method TEXT NOT NULL
    )
    ''')
    # Create transaction_items table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS transaction_items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        transaction_id INTEGER,
        item_id INTEGER,
        quantity INTEGER,
        FOREIGN KEY (transaction_id) REFERENCES transactions(id),
        FOREIGN KEY (item_id) REFERENCES items(id)
    )
    ''')

    conn.commit()
    conn.close()

def insert_item(name, price):
    """Insert a new item into the items table."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO items (name, price) VALUES (?, ?)", (name, price))
    conn.commit()
    conn.close()

def insert_transaction(transaction_id, total_amount, date, payment_method):
    """Insert a new transaction into the transactions table."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO transactions (transaction_id, total_amount, date, payment_method) VALUES (?, ?, ?, ?)",
                   (transaction_id, total_amount, date, payment_method))
    conn.commit()
    conn.close()

def insert_transaction_item(transaction_id, item_id, quantity):
    """Insert an item purchased in a transaction."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO transaction_items (transaction_id, item_id, quantity) VALUES (?, ?, ?)",
                   (transaction_id, item_id, quantity))
    conn.commit()
    conn.close()

def get_all_items():
    """Fetch all items from the items table."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM items")
    items = cursor.fetchall()
    conn.close()
    return items

# Fernet Encryption Functions
def encrypt_data(data):
    """Encrypt data using Fernet encryption."""
    return cipher_suite.encrypt(data.encode())

def decrypt_data(encrypted_data):
    """Decrypt data using Fernet decryption."""
    return cipher_suite.decrypt(encrypted_data).decode()

def show_available_items():
    """Displays the list of available stationery items from the database."""
    items = get_all_items()
    print("Available Items:")
    for idx, (item_id, name, price) in enumerate(items, start=1):
        print(f"{idx}. {name:<15} (Price: Rs. {price})")
    return {idx: name for idx, (item_id, name, _) in enumerate(items, start=1)}, {name: price for _, name, price in items}

def take_user_input(stationery_items, fixed_prices):
    """Takes user input for item selection and quantity."""
    user_items = []  # Initialize an empty list to store user's selected items
    while True:
        try:
            # Get item number from user
            item_no = int(input("Enter the item number you want to purchase (0 to finish): "))
            if item_no == 0:
                break
            if item_no not in stationery_items:
                print("Invalid item number. Please select from the available items.")
                continue
            item_name = stationery_items[item_no]
            quantity = int(input(f"Enter the quantity for {item_name}: "))
            if quantity <= 0:
                print("Quantity must be greater than 0. Try again.")
                continue
            price = fixed_prices[item_name]
            user_items.append({"name": item_name, "quantity": quantity, "price": price})
        except ValueError:
            print("Invalid input. Please enter numbers only.")
    return user_items

def create_receipt(data):
    """Creates a payment receipt as a PDF file and saves it to Google Drive."""
    os.makedirs('/content/drive/MyDrive/CipherByte Tasks/Receipts', exist_ok=True)

    filename = '/content/drive/MyDrive/CipherByte Tasks/Receipts/receipt_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.pdf'
    doc = canvas.Canvas(filename, pagesize=letter)

    page_width, page_height = letter
    border_x, border_y = 50, 50
    border_width, border_height = page_width - 100, page_height - 100

    # Draw the border box
    p = doc.beginPath()
    p.moveTo(border_x, border_y)
    p.lineTo(border_x + border_width, border_y)
    p.lineTo(border_x + border_width, border_y + border_height)
    p.lineTo(border_x, border_y + border_height)
    p.close()
    doc.rect(border_x, border_y, border_width, border_height)

    # Receipt header
    doc.setFont("Helvetica-Bold", 16)
    doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 30, "Payment Receipt")

    # Store information
    doc.setFont("Helvetica", 12)
    doc.drawString(border_x + 20, border_y + border_height - 60, f"Store Name: {data['store_name']}")
    doc.drawString(border_x + 20, border_y + border_height - 80, f"Store Address: {data['store_address']}")

    # Transaction details
    doc.setFont("Helvetica-Bold", 14)
    doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 120, "Transaction Details")
    doc.setFont("Courier", 12)
    doc.drawString(border_x + 20, border_y + border_height - 150, f"Transaction ID: {decrypt_data(data['transaction_id'])}")
    doc.drawString(border_x + 20, border_y + border_height - 170, f"Date: {data['date']}")
    doc.drawString(border_x + 20, border_y + border_height - 190, f"Total Amount: Rs. {data['total_amount']:.2f}")

    # Itemized list
    doc.setFont("Helvetica-Bold", 12)
    doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 220, "Items Purchased")
    doc.setFont("Courier", 10)

    # Right-aligning prices
    right_align_x = border_x + border_width - 100  # X position for right alignment
    y = border_y + border_height - 250
    for item in data['items']:
        subtotal = item['quantity'] * item['price']
        item_str = f"{item['quantity']} x {item['name']} @ Rs. {item['price']} each"
        doc.drawString(border_x + 20, y, item_str)
        doc.drawRightString(right_align_x, y, f"Rs. {subtotal:.2f}")
        y -= 20

    # Payment method
    doc.setFont("Helvetica-Bold", 12)
    doc.drawCentredString(border_x + border_width / 2, border_y + 100, "Payment Method")
    doc.setFont("Helvetica", 12)
    doc.drawCentredString(border_x + border_width / 2, border_y + 80, f"{data['payment_method']}")

    # Footer
    doc.setFont("Helvetica", 8)
    doc.drawCentredString(border_x + border_width / 2, border_y + 20, "This receipt is for your records only.")
    doc.save()
    print(f"Receipt saved successfully to {filename}")

# Main execution
if __name__ == "__main__":
    # Create database tables
    create_tables()
    # Mount Google Drive
    drive.mount('/content/drive')
    stationery_items, fixed_prices = show_available_items()
    user_items = take_user_input(stationery_items, fixed_prices)

    if not user_items:
        print("No items selected. Exiting.")
    else:
        total_amount = sum(item['quantity'] * item['price'] for item in user_items)
        # Encrypt transaction ID and store transaction
        encrypted_transaction_id = encrypt_data(str(random.randint(10000, 99999)))
        receipt_data = {
            "store_name": "Stationery",
            "store_address": "India",
            "transaction_id": encrypted_transaction_id,
            "date": datetime.datetime.now().strftime('%Y-%m-%d'),
            "total_amount": total_amount,
            "items": user_items,
            "payment_method": "Cash"
        }
        # Store transaction in the database
        insert_transaction(decrypt_data(encrypted_transaction_id), total_amount, receipt_data["date"], receipt_data["payment_method"])
        for item in user_items:
            item_id = [i[0] for i in get_all_items() if i[1] == item['name']][0]
            insert_transaction_item(decrypt_data(encrypted_transaction_id), item_id, item['quantity'])

        # Generate receipt
        create_receipt(receipt_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Available Items:
1. Pen             (Price: Rs. 20.0)
2. Pencil          (Price: Rs. 10.0)
3. Notebook        (Price: Rs. 75.0)
4. Eraser          (Price: Rs. 15.0)
5. Ruler           (Price: Rs. 30.0)
6. Sharpener       (Price: Rs. 20.0)
7. Marker          (Price: Rs. 40.0)
8. Glue            (Price: Rs. 45.0)
9. Stapler         (Price: Rs. 120.0)
10. Paper Clips     (Price: Rs. 7.0)
Enter the item number you want to purchase (0 to finish): 1
Enter the quantity for Pen: 2
Enter the item number you want to purchase (0 to finish): 0
Receipt saved successfully to /content/drive/MyDrive/CipherByte Tasks/Receipts/receipt_20250113_092537.pdf


In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00


In [ ]:
import sqlite3

def insert_sample_items():
    """Insert sample items into the items table."""
    sample_items = [
        ('Pen', 20),
        ('Pencil', 10),
        ('Notebook', 75),
        ('Eraser', 15),
        ('Ruler', 30),
        ('Sharpener', 20),
        ('Marker', 40),
        ('Glue', 45),
        ('Stapler', 120),
        ('Paper Clips', 7)
    ]

    # Connect to the database
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()

    # Insert sample items into the table
    cursor.executemany("INSERT INTO items (name, price) VALUES (?, ?)", sample_items)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()
    print("Sample items inserted successfully.")

# Call the function to insert the items
insert_sample_items()


Sample items inserted successfully.


In [ ]:
import sqlite3

def delete_all_items():
    """Delete all entries in the items table."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM items")
    conn.commit()  # Save the changes
    conn.close()
    print("All entries in the items table have been deleted.")

# Call the function
delete_all_items()


In [ ]:
import sqlite3

def view_items():
    """View all items in the items table."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM items")
    items = cursor.fetchall()
    conn.close()
    print("Items Table:")
    for item in items:
        print(item)

def view_transactions():
    """View all transactions in the transactions table."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM transactions")
    transactions = cursor.fetchall()
    conn.close()
    print("\nTransactions Table:")
    for transaction in transactions:
        print(transaction)

def view_transaction_items():
    """View all transaction items in the transaction_items table."""
    conn = sqlite3.connect('stationery_store.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM transaction_items")
    transaction_items = cursor.fetchall()
    conn.close()
    print("\nTransaction Items Table:")
    for transaction_item in transaction_items:
        print(transaction_item)

# Call the functions to view the data in each table
view_items()
view_transactions()
view_transaction_items()
